In [ ]:
import os
import cv2
import numpy as np

def motion_detector():
    frame_count = 0
    previous_frame = None
    image_directory = "data/1 Oude Fitness/partition 5"
    image_files = os.listdir(image_directory)

    for image_file in image_files:
        if image_file.endswith(".jpg"):
            frame_count += 1
            img_rgb = cv2.imread(os.path.join(image_directory, image_file))

            if ((frame_count % 2) == 0):
                prepared_frame = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
                prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5,5), sigmaX=0)
                if (previous_frame is None):
                    previous_frame = prepared_frame
                    continue

                diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
                previous_frame = prepared_frame

                kernel = np.ones((5, 5))
                diff_frame = cv2.dilate(diff_frame, kernel, 1)

                thresh_frame = cv2.threshold(src=diff_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]

                contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(image=img_rgb, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    if cv2.contourArea(contour) < 50:
                        # too small: skip!
                        continue
                    (x, y, w, h) = cv2.boundingRect(contour)
                    cv2.rectangle(img=img_rgb, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
                cv2.imshow('Motion detector', img_rgb)
                cv2.waitKey(0)

        if (cv2.waitKey(30) == 27):
            break
    cv2.destroyAllWindows()

motion_detector()

Detecting motion using optical flow.

In [ ]:
image_directory = "data/1 Oude Fitness/partition 5"

prev_frame = cv2.imread(os.path.join(image_directory, os.listdir(image_directory)[0]))
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

flow_image = np.zeros_like(prev_frame)

lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

prev_points = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30)

for image_path in os.listdir(image_directory):
    if image_path.endswith(".jpg"):
        frame = cv2.imread(os.path.join(image_directory, image_path))

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        curr_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None, **lk_params)

        good_points = curr_points[status == 1]
        prev_points = good_points.reshape(-1, 1, 2)

        for i, (new, old) in enumerate(zip(good_points, prev_points)):
            a, b = new.ravel()
            c, d = old.ravel()
            flow_image = cv2.line(flow_image, (int(a), int(b)), (int(c), int(d)), [0, 0, 255], 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, [0, 0, 255], -1)

        prev_gray = gray

        cv2.imshow("Optical Flow", flow_image)
        cv2.imshow("Frame", frame)

        key = cv2.waitKey(0)

        if key == ord("q"):
            break

cv2.destroyAllWindows()